In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans

In [6]:
g = nx.read_edgelist("./vidal_tsv")
g = nx.convert_node_labels_to_integers(g)

In [7]:
def group_belonging(i: int, n: list):
    if i >= sum(n):
        raise ValueError("Such item does not belong in n")
        
    for j in range(1, len(n)):
        if sum(n[:j]) > i:
            return j-1
    return len(n)-1


def generate_weights_matrix(G: nx.Graph, W: np.array, n: list) -> np.array:
    ''''''
    
    #cumulative_n = [sum(n[:i]) for i in range(len(n)+1)]
    
    weights_matrix = np.full((sum(n), sum(n)), -1.0)

    #For each node:
    for key in range(sum(n)):
        group1 = group_belonging(key, n)

        #For each other node:
        for potential_neighbour in range(sum(n)):

            #If the node is a neighbour:
            if (potential_neighbour in G.adj[key]):

                #If this is the first time we see the edge - generate weight:
                if potential_neighbour > key:
                    group2 = group_belonging(potential_neighbour, n)
                    mean = W[group1][group2] #Retrieve the mean of the distribution from the weight matrix
                    weight = round(abs(np.random.normal(loc=mean, scale=1.0)), 2) #Using the absolute value here
                    weights_matrix[key][potential_neighbour] = weight

                #Else, copy the weight given before:
                else:
                    weights_matrix[key][potential_neighbour] = weights_matrix[potential_neighbour][key]

            elif potential_neighbour == key: #Self-loops' weight goes here
                weights_matrix[key][potential_neighbour] = 0

            else: #Regular disconnected nodes
                weights_matrix[key][potential_neighbour] = 0
                
    return weights_matrix

In [5]:
np.random.random((5, 5))

array([[0.36119547, 0.12300162, 0.3357875 , 0.20030531, 0.68590409],
       [0.23884087, 0.1523217 , 0.53237398, 0.41053776, 0.84537043],
       [0.77038629, 0.37564969, 0.33464693, 0.55497221, 0.08277426],
       [0.89723809, 0.39282269, 0.57409779, 0.13222192, 0.56125917],
       [0.42238598, 0.57712705, 0.44474806, 0.93270828, 0.19383199]])

In [20]:
k = 4
P = np.random.random((k, k)) / 2 
P = P + np.diag(np.random.random(k) / 4)
# P = (P + P.transpose()) / 2
P

array([[0.52429769, 0.38362607, 0.37776278, 0.26015334],
       [0.31708468, 0.50727901, 0.18707806, 0.46227961],
       [0.19918515, 0.20772215, 0.52248608, 0.25637579],
       [0.39914289, 0.28225248, 0.04421757, 0.39379054]])

In [24]:
np.random.random(k)
np.diag(np.random.random(k)/4)
P, np.around(P, 2)

(array([[0.52, 0.38, 0.38, 0.26],
        [0.32, 0.51, 0.19, 0.46],
        [0.2 , 0.21, 0.52, 0.26],
        [0.4 , 0.28, 0.04, 0.39]]),
 array([[0.52, 0.38, 0.38, 0.26],
        [0.32, 0.51, 0.19, 0.46],
        [0.2 , 0.21, 0.52, 0.26],
        [0.4 , 0.28, 0.04, 0.39]]))

In [8]:
def generate_dataset_and_test_spectral(k: int, n = None):
    '''Use this to generate ground-truth data and test Spectral Clustering using this dataset
    
    I am not entire sure this last part works correctly (but the U is generated perfectly)'''
    
    if n is None:
        n = [random.randint(10, 50) for _ in range(k)]
        print("n =", n, ", sum of n =", sum(n))
        
    #This is ingenious. 
    # All inter-community probs for edge are [0, 1/2] while in-community probs are [1/4, 3/4]
    P = np.random.random((k, k))/2 #([0,0.5])
    P = P + np.diag(np.random.random(k)/4)
    #Avering out the off-diagonal entries, 'cause P must be symmetrical. Also, rounding
    P = (P + P.transpose()) / 2
    P = np.around(P, 2)
    
    #All inter-community Gaussian means of weights are [1/4, 3/4] while in-community means are [1/2, 1]
    W = P + np.full((k, k), 0.25)
    
    #Create the graph
    G = nx.stochastic_block_model(n, P)
    
    #Generate a weight (Gaussian but positive) for each edge on the graph
    weights_matrix = generate_weights_matrix(G, W, n)
    
    #Generate the matrix of degrees of each node
    D = np.diag(np.sum(weights_matrix, axis=1))
    
    #L is the laplacian (unnormalized)
    L = D - weights_matrix
    
    #Extract the eigenstuff from the unnormalized Laplacian
    eigenvalues, eigenvectors = np.linalg.eig(L)
    
    #Sort the eigenstuff by the eigenvalues (ascending)
    eigenstuff = dict(zip(eigenvalues, eigenvectors))
    eigenstuff = dict(sorted(eigenstuff.items()))
    
    #Extract the k lowest eigenstuff
    first_k_eigenstuff, i = {}, 0
    for key in eigenstuff:
        first_k_eigenstuff[key] = eigenstuff[key]
        i += 1
        if i >= k:
            break

    #At this point we can get rid of eigenvalues and just focus on creating matrix U with eigenvectors as columns
    U = np.column_stack([first_k_eigenstuff[key] for key in first_k_eigenstuff])
    
    #This already works per row, so we do not need to reshape the data again :D
    kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(U)
    
    y_true = []
    for cluster_number in range(len(n)):
        y_true += [cluster_number for _ in range(n[cluster_number])]
    
    return kmeans

In [9]:
kmeans = generate_dataset_and_test_spectral(k=4)

n = [30, 20, 31, 36] , sum of n = 117


In [10]:
print(len(kmeans.labels_)) #if it matches sum(n), then everything's great
kmeans.labels_

117


array([2, 0, 3, 2, 2, 3, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 3, 1, 3, 2, 2,
       3, 3, 1, 2, 1, 3, 0, 3, 3, 3, 3, 1, 3, 3, 3, 3, 2, 1, 2, 2, 2, 2,
       0, 2, 2, 0, 2, 3, 2, 0, 1, 3, 1, 2, 3, 3, 2, 3, 2, 3, 2, 2, 3, 0,
       3, 3, 1, 3, 3, 0, 2, 2, 2, 0, 1, 3, 2, 1, 2, 2, 3, 3, 0, 3, 3, 2,
       3, 1, 3, 0, 1, 2, 3, 2, 3, 2, 2, 3, 2, 1, 2, 0, 2, 2, 1, 1, 3, 2,
       2, 3, 2, 0, 2, 1, 2], dtype=int32)